# Intelligent Agents: Reflex-Based Agents for the Vacuum-cleaner World

Student Name: Davis Lynn

I have used the following AI tools: ChatGPT for Debugging Environment Code

I understand that my submission needs to be my own work: DPL

## Learning Outcomes

* Design and build a simulation environment that models sensor inputs, actuator effects, and performance measurement.
* Apply core AI concepts by implementing the agent function for a simple and model-based reflex agents that respond to environmental percepts.
* Practice how the environment and the agent function interact.
* Analyze agent performance through controlled experiments across different environment configurations.
* Graduate Students: Develop strategies for handling uncertainty and imperfect information in autonomous agent systems.

## Instructions

Total Points: Undergrads 98 + 5 bonus / Graduate students 110

Complete this notebook. Use the provided notebook cells and insert additional code and markdown cells as needed. Submit the completely rendered notebook as a HTML file.

### AI Use

Here are some guidelines that will make it easier for you:

* __Don't:__ Rely on AI auto completion. You will waste a lot of time trying to figure out how the suggested code relates to what we do in class. Turn off AI code completion (e.g., Copilot) in your IDE.
* __Don't:__ Do not submit code/text that you do not understand or have not checked to make sure that it is complete and correct.
* __Do:__ Use AI for debugging and letting it explain code and concepts from class.

### Using Visual Studio Code

If you use VS code then you can use `Export` (click on `...` in the menu bar) to save your notebook as a HTML file. Note that you have to run all blocks before so the HTML file contains your output.

### Using Google Colab

In Colab you need to save the notebook on GoogleDrive to work with it. For this you need to mount your google dive and change to the correct directory by uncommenting the following lines and running the code block.

In [2]:
# from google.colab import drive
# import os
#
# drive.mount('/content/drive')
# os.chdir('/content/drive/My Drive/Colab Notebooks/')

Once you are done with the assignment and have run all code blocks using `Runtime/Run all`, you can convert the file on your GoogleDrive into HTML be uncommenting the following line and running the block.

In [3]:
# %jupyter nbconvert --to html Copy\ of\ robot_vacuum.ipynb

You may have to fix the file location or the file name to match how it looks on your GoogleDrive. You can navigate in Colab to your GoogleDrive using the little folder symbol in the navigation bar to the left.

## Introduction

In this assignment you will implement a simulator environment for an automatic vacuum cleaner robot, a set of different reflex-based agent programs, and perform a comparison study for cleaning a single room. Focus on the __cleaning phase__ which starts when the robot is activated and ends when the last dirty square in the room has been cleaned. Someone else will take care of the agent program needed to navigate back to the charging station after the room is clean.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to clean the whole room.

__Environment:__ A room with $n \times n$ squares where $n = 5$. Dirt is randomly placed on each square with probability $p = 0.2$. For simplicity, you can assume that the agent knows the size and the layout of the room (i.e., it knows $n$). To start, the agent is placed on a random square.

__Actuators:__ The agent can clean the current square (action `suck`) or move to an adjacent square by going `north`, `east`, `south`, or `west`.

__Sensors:__ Four bumper sensors, one for north, east, south, and west; a dirt sensor reporting dirt in the current square.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent:  

In [7]:
import numpy as np

actions = ["north", "east", "west", "south", "suck"]

def simple_randomized_agent(bumpers, dirty):
    return np.random.choice(actions)

In [8]:
# define percepts (current location is NW corner and it is dirty)
bumpers = {"north" : True, "east" : False, "south" : False, "west" : True}
dirty = True

# call agent program function with percepts and it returns an action
simple_randomized_agent(bumpers, dirty)

np.str_('suck')

__Note:__ This is not a rational intelligent agent. It ignores its sensors and may bump into a wall repeatedly or not clean a dirty square. You will be asked to implement rational agents below.

## Simple environment example

We implement a simple simulation environment that supplies the agent with its percepts.
The simple environment is infinite in size (bumpers are always `False`) and every square is always dirty, even if the agent cleans it. The environment function returns a different performance measure than the one specified in the PEAS description! Since the room is infinite and all squares are constantly dirty, the agent can never clean the whole room. Your implementation needs to implement the **correct performance measure.** The energy budget of the agent is specified as `max_steps`.

In [6]:
def simple_environment(agent_function, max_steps, verbose = True):
    num_cleaned = 0

    for i in range(max_steps):
        dirty = True
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}

        action = agent_function(bumpers, dirty)
        if (verbose): print("step", i , "- action:", action)

        if (action == "suck"):
            num_cleaned = num_cleaned + 1

    return num_cleaned



Do one simulation run with a simple randomized agent that has enough energy for 20 steps.

In [7]:
simple_environment(simple_randomized_agent, max_steps = 20)

step 0 - action: north
step 1 - action: south
step 2 - action: east
step 3 - action: suck
step 4 - action: suck
step 5 - action: suck
step 6 - action: suck
step 7 - action: west
step 8 - action: suck
step 9 - action: east
step 10 - action: suck
step 11 - action: suck
step 12 - action: west
step 13 - action: north
step 14 - action: south
step 15 - action: north
step 16 - action: suck
step 17 - action: west
step 18 - action: south
step 19 - action: west


8

# Tasks

## General [10 Points]

1. Make sure that you use the latest version of this notebook.
2. Your implementation can use libraries like math, numpy, scipy, but not libraries that implement intelligent agents or complete search algorithms. Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design.
3. You notebook needs to be formatted professionally.
    - Add additional markdown blocks for your description, comments in the code, add tables and use mathplotlib to produce charts where appropriate
    - Do not show debugging output or include an excessive amount of output.
    - Check that your submitted file is readable and contains all figures.
4. Document your code. Use comments in the code and add a discussion of how your implementation works and your design choices.


## Task 1: Implement a simulation environment [20 Points]

The simple environment above is not very realistic. Your environment simulator needs to follow the PEAS description from above. It needs to:

* Initialize the environment by storing the state of each square (clean/dirty) and making some dirty. ([Help with random numbers and arrays in Python](https://github.com/mhahsler/CS7320-AI/blob/master/HOWTOs/random_numbers_and_arrays.ipynb))
* Keep track of the agent's position.
* Call the agent function repeatedly and provide the agent function with the sensor inputs.  
* React to the agent's actions. E.g, by removing dirt from a square or moving the agent around unless there is a wall in the way.
* Keep track of the performance measure. That is, track the agent's actions until all dirty squares are clean and count the number of actions it takes the agent to complete the task.

The easiest implementation for the environment is to hold an 2-dimensional array to represent if squares are clean or dirty and to call the agent function in a loop until all squares are clean or a predefined number of steps have been reached (i.e., the robot runs out of energy).

The simulation environment should be a function like the `simple_environment()` and needs to work with the simple randomized agent program from above. **Use the same environment for all your agent implementations in the tasks below.**

*Note on debugging:* Debugging is difficult. Make sure your environment prints enough information when you use `verbose = True`. Also, implementing a function that the environment can use to displays the room with dirt and the current position of the robot at every step is very useful.  

In [3]:
import pandas as pd
import numpy as np

# W = Wall
# 1 = Dirt
# 0 = Clean
# R = Robot

#  generating dataframe of size n
def create_empty_env(n):
    mydf = pd.DataFrame(np.zeros((n, n), dtype=object))

    # add walls
    mydf.iloc[0, :]  = "W"
    mydf.iloc[-1, :] = "W"
    mydf.iloc[:, 0]  = "W"
    mydf.iloc[:, -1] = "W"
    return mydf


#  function to randomly place dirt in the environment
def place_dirt(mydf,n):
    prob=0.2
    numdirty = 0
    for i in range(1, n-1):
        for j in range(1, n-1):
            if np.random.rand() <= prob:
                mydf.iat[i, j] = 1
                numdirty = numdirty + 1
    return mydf, numdirty


# function to randomly place robot in the environment (not physicaly in the DF, only printed with output)
def place_robot(n, robot_x, robot_y):
    i = np.random.randint(1, n-1)
    j = np.random.randint(1, n-1)
    robot_x, robot_y = i, j
    return robot_x, robot_y

# function to suck dirt
def suck(mydf, robot_x, robot_y):
    mydf.iat[robot_x, robot_y] = 0
    return mydf

# function to move robot besed on direction string
def action(mydf, robot_x, robot_y, n , command, energy):
    if command == "north" and robot_x > 1:
        robot_x = robot_x - 1
    elif command == "south" and robot_x < n-2:
        robot_x = robot_x + 1
    elif command == "west" and robot_y > 1:
        robot_y = robot_y - 1
    elif command == "east" and robot_y < n-2:
        robot_y = robot_y + 1
    elif command == "suck":
        suck(mydf, robot_x, robot_y)
    elif command == "None":
        return robot_x, robot_y, energy
    energy = energy+1
    return robot_x, robot_y, energy


# function to check sensors and return bumpers and dirty status
def check_sensors(mydf, robot_x, robot_y, bumpers, dirty):
    if mydf.iat[robot_x-1, robot_y] == "W":
        bumpers["north"] = True
    if mydf.iat[robot_x+1, robot_y] == "W":
        bumpers["south"] = True
    if mydf.iat[robot_x, robot_y-1] == "W":
        bumpers["west"] = True
    if mydf.iat[robot_x, robot_y+1] == "W":
        bumpers["east"] = True
    dirty = (mydf.iat[robot_x, robot_y] == 1)
    return bumpers, dirty

# prints environment with robot position
def print_env(mydf, robot_x, robot_y):
    df_copy = mydf.copy()
    df_copy.iat[robot_x, robot_y] = "R"
    print(df_copy)
    print("Robot position: (", robot_x, ",", robot_y, ")")
    return

# main simulation environment
def simulation_environment(n, agent, verbose=True):
    mydf = create_empty_env(n)
    mydf, numdirty = place_dirt(mydf, n)   # fixed order
    robot_x, robot_y = place_robot(n, 0, 0)
    num_cleaned = 0
    energy = 0
    i = 1
    if verbose: print_env(mydf, robot_x, robot_y)

    while num_cleaned < numdirty:
        bumpers = {"north": False, "south": False, "west": False, "east": False}
        bumpers, dirty = check_sensors(mydf, robot_x, robot_y, bumpers, False)  # pass args

        action_str = agent(bumpers, dirty)
        robot_x, robot_y, energy = action(mydf, robot_x, robot_y, n, action_str, energy)

        if action_str == "suck" and dirty:
            num_cleaned += 1

        if verbose:
            print("step", i, "- action:", action_str)
            print_env(mydf, robot_x, robot_y)
        i += 1

    return num_cleaned, energy



Show that your environment works with the simple randomized agent from above.

In [9]:
num_cleaned, energy = simulation_environment(7, simple_randomized_agent)
print("Number of cleaned locations:", num_cleaned)
print("Energy Used:", energy)

   0  1  2  3  4  5  6
0  W  W  W  W  W  W  W
1  W  0  1  0  0  0  W
2  W  0  0  0  1  0  W
3  W  R  0  0  0  0  W
4  W  0  0  1  0  1  W
5  W  0  0  1  0  0  W
6  W  W  W  W  W  W  W
step 1 - action: suck
   0  1  2  3  4  5  6
0  W  W  W  W  W  W  W
1  W  0  1  0  0  0  W
2  W  0  0  0  1  0  W
3  W  R  0  0  0  0  W
4  W  0  0  1  0  1  W
5  W  0  0  1  0  0  W
6  W  W  W  W  W  W  W
step 2 - action: west
   0  1  2  3  4  5  6
0  W  W  W  W  W  W  W
1  W  0  1  0  0  0  W
2  W  0  0  0  1  0  W
3  W  R  0  0  0  0  W
4  W  0  0  1  0  1  W
5  W  0  0  1  0  0  W
6  W  W  W  W  W  W  W
step 3 - action: west
   0  1  2  3  4  5  6
0  W  W  W  W  W  W  W
1  W  0  1  0  0  0  W
2  W  0  0  0  1  0  W
3  W  R  0  0  0  0  W
4  W  0  0  1  0  1  W
5  W  0  0  1  0  0  W
6  W  W  W  W  W  W  W
step 4 - action: east
   0  1  2  3  4  5  6
0  W  W  W  W  W  W  W
1  W  0  1  0  0  0  W
2  W  0  0  0  1  0  W
3  W  0  R  0  0  0  W
4  W  0  0  1  0  1  W
5  W  0  0  1  0  0  W
6  W  W  W  W  

## Task 2:  Implement a simple reflex agent [10 Points]

The simple reflex agent randomly walks around but reacts to the bumper sensor by not bumping into the wall and to dirt with sucking. Implement the agent program as a function.

_Note:_ Agents cannot directly use variable in the environment. They only gets the percepts as the arguments to the agent function. Use the function signature for the `simple_randomized_agent` function above.

In [ ]:
actions = ["north", "east", "west", "south", "suck"]
def simple_reflex_agent(bumpers, dirty):
    while True: #loops until action is chosen
        action = np.random.choice(actions) # pick random action
        if dirty and action == "suck": # if dirty, suck
            return action
        elif bumpers["north"]==False and action == "north": # if no wall north, go north
            return action
        elif bumpers["south"]==False and action == "south": # if no wall south, go south
            return action
        elif bumpers["west"]==False and action == "west": # if no wall west, go west
            return action
        elif bumpers["east"]==False and action == "east": # if no wall east, go east
            return action
    

Show how the agent works with your environment.

In [19]:
num_cleaned, energy = simulation_environment(7, simple_reflex_agent)
print("Number of cleaned locations:", num_cleaned)
print("Energy Used:", energy)

   0  1  2  3  4  5  6
0  W  W  W  W  W  W  W
1  W  0  0  1  0  0  W
2  W  0  0  0  0  0  W
3  W  0  0  0  0  0  W
4  W  0  1  0  0  0  W
5  W  0  1  R  1  0  W
6  W  W  W  W  W  W  W
Robot position: ( 5 , 3 )
step 1 - action: west
   0  1  2  3  4  5  6
0  W  W  W  W  W  W  W
1  W  0  0  1  0  0  W
2  W  0  0  0  0  0  W
3  W  0  0  0  0  0  W
4  W  0  1  0  0  0  W
5  W  0  R  0  1  0  W
6  W  W  W  W  W  W  W
Robot position: ( 5 , 2 )
step 2 - action: north
   0  1  2  3  4  5  6
0  W  W  W  W  W  W  W
1  W  0  0  1  0  0  W
2  W  0  0  0  0  0  W
3  W  0  0  0  0  0  W
4  W  0  R  0  0  0  W
5  W  0  1  0  1  0  W
6  W  W  W  W  W  W  W
Robot position: ( 4 , 2 )
step 3 - action: east
   0  1  2  3  4  5  6
0  W  W  W  W  W  W  W
1  W  0  0  1  0  0  W
2  W  0  0  0  0  0  W
3  W  0  0  0  0  0  W
4  W  0  1  R  0  0  W
5  W  0  1  0  1  0  W
6  W  W  W  W  W  W  W
Robot position: ( 4 , 3 )
step 4 - action: north
   0  1  2  3  4  5  6
0  W  W  W  W  W  W  W
1  W  0  0  1  0  0  W
2

## Task 3: Implement a model-based reflex agent [20 Points]

Model-based agents use a state to keep track of what they have done and perceived so far. Your agent needs to find out where it is located and then keep track of its current location. You also need a set of rules based on the state and the percepts to make sure that the agent will clean the whole room. For example, the agent can move to a corner to determine its location and then it can navigate through the whole room and clean dirty squares.

Describe how you define the __agent state__ and how your agent works before implementing it. ([Help with implementing state information on Python](https://github.com/mhahsler/CS7320-AI/blob/master/HOWTOs/store_agent_state_information.ipynb))

### Task 3 Answer
My model has two stages of traversing the room. In the first state, the robot traverses west and then north to find the (1,1) coordinate in the room. Once the robot makes it to the top corner of the room, the robot switches to state 2, which sweeps the room in a 'Snake' like pattern until it grabs the last dirt on the page. The following images describe the behavior visually:
![image](pictures\State1State2.png)

This is an imperfect solution, as it must traverse the State 1 path twice. In a future version of this model, I would like to save what locations the robot has been to already, and not make the robot traverse those locations twice. This would lead to a solution that would lead to nxn(sideways movement) + n(downward movement) + num_dirt energy used, which is optimal in a square room. 

In [ ]:
known_x, known_y = 0, 0   # 0,0 means we haven't reached NW corner yet
e_w = 0                   # 0 = moving east, 1 = moving west

def state_1(bumpers, dirty): # seeking NW corner
    global known_x, known_y, e_w
    # Move to west wall first
    if dirty:
        return "suck"
    if not bumpers["west"]:
        return "west"
    # Then move to north wall
    if not bumpers["north"]:
        return "north"
    # At NW corner: switch to sweep mode
    known_x, known_y = 1, 1
    e_w = 0  # start going east
    return "east"

def state_2(bumpers, dirty): # sweeping room
    global e_w
    if dirty:
        return "suck"

    if e_w == 0:  # moving east
        if not bumpers["east"]:
            return "east"
        # hit east wall: turn around and go down a row if possible
        e_w = 1
        if not bumpers["south"]:
            return "south"
        # nowhere to go south (bottom edge) — just stay put
        return "west" 
    else:  # e_w == 1, moving west
        if not bumpers["west"]:
            return "west"
        # hit west wall: turn around and go down a row if possible
        e_w = 0
        if not bumpers["south"]:
            return "south"
        return "east"  

def model_based_agent(bumpers, dirty):
    global known_x, known_y
    # If we haven't found the NW corner yet, keep seeking it.
    if known_x == 0 and known_y == 0:
        return state_1(bumpers, dirty)
    # Otherwise, sweep.
    return state_2(bumpers, dirty)

def reset_globals(): #reset global variables for multiple runs
    global known_x, known_y, e_w
    known_x, known_y = 0, 0
    e_w = 0


Show how the agent works with your environment.

In [ ]:
reset_globals()
num_cleaned, energy = simulation_environment(7, model_based_agent)
print("Number of cleaned locations:", num_cleaned)
print("Energy Used:", energy)

   0  1  2  3  4  5  6
0  W  W  W  W  W  W  W
1  W  0  0  0  0  0  W
2  W  0  0  0  R  0  W
3  W  0  0  0  0  0  W
4  W  0  0  0  0  0  W
5  W  1  0  0  0  0  W
6  W  W  W  W  W  W  W
Robot position: ( 2 , 4 )
step 1 - action: west
   0  1  2  3  4  5  6
0  W  W  W  W  W  W  W
1  W  0  0  0  0  0  W
2  W  0  0  R  0  0  W
3  W  0  0  0  0  0  W
4  W  0  0  0  0  0  W
5  W  1  0  0  0  0  W
6  W  W  W  W  W  W  W
Robot position: ( 2 , 3 )
step 2 - action: west
   0  1  2  3  4  5  6
0  W  W  W  W  W  W  W
1  W  0  0  0  0  0  W
2  W  0  R  0  0  0  W
3  W  0  0  0  0  0  W
4  W  0  0  0  0  0  W
5  W  1  0  0  0  0  W
6  W  W  W  W  W  W  W
Robot position: ( 2 , 2 )
step 3 - action: west
   0  1  2  3  4  5  6
0  W  W  W  W  W  W  W
1  W  0  0  0  0  0  W
2  W  R  0  0  0  0  W
3  W  0  0  0  0  0  W
4  W  0  0  0  0  0  W
5  W  1  0  0  0  0  W
6  W  W  W  W  W  W  W
Robot position: ( 2 , 1 )
step 4 - action: north
   0  1  2  3  4  5  6
0  W  W  W  W  W  W  W
1  W  R  0  0  0  0  W
2 

## Task 4: Simulation study [30 Points]

Compare the performance (the performance measure is defined in the PEAS description above) of the agents using  environments of different size. Do at least $5 \times 5$, $10 \times 10$ and
$100 \times 100$. Use 100 random runs for each. Present the results using tables and graphs. Discuss the differences between the agents.
([Help with charts and tables in Python](https://github.com/mhahsler/CS7320-AI/blob/master/HOWTOs/charts_and_tables.ipynb))

In [ ]:
#initializing empty lists to store results
five_random = []
ten_random = []
hundred_random = []

five_reflex = []
ten_reflex = []
hundred_reflex = []

five_model = []
ten_model = []
hundred_model = []

#running 100 trials for each environment size and agent type
for i in range(100):
    print("Trial:", i+1)
    print("Randomized Agent")
    #print("5x5 Environment")
    num_cleaned, energy = simulation_environment(7, simple_randomized_agent, verbose=False)
    five_random.append(energy)
    #print("10x10 Environment")
    num_cleaned, energy = simulation_environment(12, simple_randomized_agent, verbose=False)
    ten_random.append(energy)
    #print("100x100 Environment")
    num_cleaned, energy = simulation_environment(102, simple_randomized_agent, verbose=False)
    hundred_random.append(energy)
    print("Reflex Agent")
    #print("5x5 Environment")
    num_cleaned, energy = simulation_environment(7, simple_reflex_agent, verbose=False)
    five_reflex.append(energy)
    #print("10x10 Environment")
    num_cleaned, energy = simulation_environment(12, simple_reflex_agent, verbose=False)
    ten_reflex.append(energy)
    #print("100x100 Environment")
    num_cleaned, energy = simulation_environment(102, simple_reflex_agent, verbose=False)
    hundred_reflex.append(energy)
    print("Model-Based Agent")
    #print("5x5 Environment")
    num_cleaned, energy = simulation_environment(7, model_based_agent, verbose=False)
    five_model.append(energy)
    reset_globals()
    #print("10x10 Environment")
    num_cleaned, energy = simulation_environment(12, model_based_agent, verbose=False)
    ten_model.append(energy)
    reset_globals()
    #print("100x100 Environment")
    num_cleaned, energy = simulation_environment(102, model_based_agent, verbose=False)
    hundred_model.append(energy)
    reset_globals()



# Calculating averages
data = {
    "Environment Size": ["5x5", "10x10", "100x100"],
    "Randomized Agent": [
        sum(five_random) / len(five_random),
        sum(ten_random) / len(ten_random),
        sum(hundred_random) / len(hundred_random) if hundred_random else "N/A"
    ],
    "Simple Reflex Agent": [
        sum(five_reflex) / len(five_reflex),
        sum(ten_reflex) / len(ten_reflex),
        sum(hundred_reflex) / len(hundred_reflex) if hundred_reflex else "N/A"
    ],
    "Model-based Reflex Agent": [
        sum(five_model) / len(five_model),
        sum(ten_model) / len(ten_model),
        sum(hundred_model) / len(hundred_model) if hundred_model else "N/A"
    ]
}

df_results = pd.DataFrame(data)

print(df_results)

Trial: 1
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 2
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 3
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 4
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 5
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 6
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 7
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 8
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 9
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 10
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 11
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 12
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 13
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 14
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 15
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 16
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 17
Randomized Agent
Reflex Agent
Model-Based Agent
Trial: 18
Randomized Ag

In [ ]:
#print arrays to csv
five = np.array(five_random)
np.savetxt("five_random.csv", five, delimiter=",")
ten = np.array(ten_random)
np.savetxt("ten_random.csv", ten, delimiter=",")
hundred = np.array(hundred_random)
np.savetxt("hundred_random.csv", hundred, delimiter=",")
five = np.array(five_reflex)
np.savetxt("five_reflex.csv", five, delimiter=",")
ten = np.array(ten_reflex)
np.savetxt("ten_reflex.csv", ten, delimiter=",")
hundred = np.array(hundred_reflex)
np.savetxt("hundred_reflex.csv", hundred, delimiter=",")
five = np.array(five_model)
np.savetxt("five_model.csv", five, delimiter=",")
ten = np.array(ten_model)
np.savetxt("ten_model.csv", ten, delimiter=",")
hundred = np.array(hundred_model)
np.savetxt("hundred_model.csv", hundred, delimiter=",")



Fill out the following table with the average performance measure for 100 random runs (you may also create this table with code):

| Size     | Randomized Agent | Simple Reflex Agent | Model-based Reflex Agent |
|----------|------------------|---------------------|--------------------------|
| 5x5     | 413.41|270.02|27.86|
| 10x10   |3080.08|2192.63 |123.32|
| 100x100 |831221.18 |689087.55|12088.77|

Add charts to compare the performance of the different agents.


### Response
![image](pictures\5numsum.png)

As we can see, as the models increased in complexity, the standard deviation and mean of the agent's energy use decreased. This can be explained by the decrease in extraneous energy usage that the robots use on actions that are not relevant to their purpose. For example, the random robot will run into walls repeatedly, whereas the reflex will not, saving energy. This is further improved by the Model based agent that tries to minimize the number of repeated cells crossed over. Overall, shaping the model's behavior towards an optimal solution will be significanty more efficient than letting a robot randomly partake in actions. This can be further represented by the follow box and whisker plot: 
![image](pictures\Box+Whisker.png)

As we can see once again, the variance is significantly higher for that of the first two models than that of the third, with many outliers lying above the Q3 value, and very few outliers below Q1. This means that the overall efficiency of the reflex or random model will not provide the benefits of an algorithm like Quicksort, which is faster in most cases and has a potential for a slower outcome, the algorithms nearly always will provide the slower outcome. 

## Task 5: Robustness of the agent implementations [10 Points]

Describe how **your agent implementations** will perform

* if it is put into a rectangular room with unknown size,
* if the cleaning area can have an irregular shape (e.g., a hallway connecting two rooms), or
* if the room contains obstacles (i.e., squares that it cannot pass through and trigger the bumper sensors).
* if the dirt sensor is not perfect and gives 10% of the time a wrong reading (clean when it is dirty or dirty when it is clean).
* if the bumper sensor is not perfect and 10% of the time does not report a wall when there is one.

### Irregular Size
- My agent was designed for a square room, but it would be able to perform in a rectangular room as well. It is designed to search for far walls, so it should perform the same

### Irrgular Shape
- My agent would not be able to handle an irregularly shaped room. It is designed to search for walls in a straight line, so it would leave dirty patches for the walls that it does not initially find.

### Obstacles
- My agent would not be able to handle obstacles in the room. It is designed to path in a straight line and cannot go around blocks, thus it would leave dirt on the room that it cannot reach beyond the obstacles.

### Dirt Sensor Failure
- My agent would not adapt for the dirt sensor failing. This would lead to a higher energy usage if the sensor gives false positives, and it could potentially leave the room dirty because it is not programmed to do multiple sweeps of the room to account for false negatives.

### Bumper Sensor Failure
- My agent would adapt to the bumper sensor failing. If a bumper gives a false negative, my robot will attempt to move in that direction, but it will just run into a wall. It will make the robot less efficient in terms of energy, but it will not stop it from cleaning the room eventually. 


## Advanced task: Imperfect Dirt Sensor

* __Graduate students__ need to complete this task [10 points]
* __Undergraduate students__ can attempt this as a bonus task [max +5 bonus points].

1. Change your simulation environment to run experiments for the following problem: The dirt sensor has a 10% chance of giving the wrong reading. Perform experiments to observe how this changes the performance of the three implementations. Your model-based reflex agent is likely not able to clean the whole room, so you need to measure performance differently as a tradeoff between energy cost and number of uncleaned squares.

2. Design an implement a solution for your model-based agent that will clean better. Show the improvement with experiments.

In [4]:
#updated environment with 10% sensor failure rate
def updated_check_sensors(mydf, robot_x, robot_y, bumpers, dirty):
    if mydf.iat[robot_x-1, robot_y] == "W":
        bumpers["north"] = True
    if mydf.iat[robot_x+1, robot_y] == "W":
        bumpers["south"] = True
    if mydf.iat[robot_x, robot_y-1] == "W":
        bumpers["west"] = True
    if mydf.iat[robot_x, robot_y+1] == "W":
        bumpers["east"] = True
    dirty = (mydf.iat[robot_x, robot_y] == 1)
    if np.random.rand() < 0.1 and not dirty:
        dirty = True
        return bumpers, dirty
    elif np.random.rand() < 0.1 and dirty:
        dirty = False
        return bumpers, dirty
    else :
        return bumpers, dirty
    
def updated_environment(n, agent, verbose=True):
    mydf = create_empty_env(n)
    mydf, numdirty = place_dirt(mydf, n)   # fixed order
    robot_x, robot_y = place_robot(n, 0, 0)
    num_cleaned = 0
    num_visited = 1
    energy = 0
    i = 1
    if verbose: print_env(mydf, robot_x, robot_y)

    while num_visited < n**2:
        bumpers = {"north": False, "south": False, "west": False, "east": False}
        bumpers, dirty = updated_check_sensors(mydf, robot_x, robot_y, bumpers, False)

        action_str = agent(bumpers, dirty)
        robot_x, robot_y, energy = action(mydf, robot_x, robot_y, n, action_str, energy)

        if action_str == "suck" and dirty:
            num_cleaned += 1
        # count unique visited locations
        num_visited = len(set((robot_x, robot_y) for _ in range(num_visited + 1)))
        if verbose:
            print("step", i, "- action:", action_str)
            print_env(mydf, robot_x, robot_y)
        i += 1

    return num_cleaned, energy


In [ ]:
num_cleaned, energy = updated_environment(7, simple_randomized_agent, verbose=False)
print("Number of cleaned locations:", num_cleaned)
print("Energy Used:", energy)

## More Advanced Implementation (not for credit)

If the assignment was to easy for yuo then you can think about the following problems. These problems are challenging and not part of this assignment. We will learn implementation strategies and algorithms useful for these tasks during the rest of the semester.

* __Obstacles:__ Change your simulation environment to run experiments for the following problem: Add random obstacle squares that also trigger the bumper sensor. The agent does not know where the obstacles are. Perform experiments to observe how this changes the performance of the three implementations. Describe what would need to be done to perform better with obstacles. Add code if you can.

* __Agent for and environment with obstacles:__ Implement an agent for an environment where the agent does not know how large the environment is (we assume it is rectangular), where it starts or where the obstacles are. An option would be to always move to the closest unchecked/uncleaned square (note that this is actually depth-first search).

* __Utility-based agent:__ Change the environment for a $5 \times 5$ room, so each square has a fixed probability of getting dirty again. For the implementation, we give the environment a 2-dimensional array of probabilities. The utility of a state is defined as the number of currently clean squares in the room. Implement a utility-based agent that maximizes the expected utility over one full charge which lasts for 100000 time steps. To do this, the agent needs to learn the probabilities with which different squares get dirty again. This is very tricky!

In [ ]:
# Your ideas/code